In [19]:
import os
import sys

In [ ]:
sys.path.append('/kaggle/input/birdclef2025-code/main_folder/')
sys.path.append('/kaggle/input/birdclef2025-env/kaggle_env/')

In [ ]:
from hydra import initialize, compose
from omegaconf import OmegaConf

# Config manager

In [20]:
CFG_PATH = "../input/birdclef2025-code/main_folder/src/config"
with initialize(config_path=CFG_PATH): 
    cfg = compose(config_name="config")  

# print(OmegaConf.to_container(cfg, resolve=False))

/tmp/ipykernel_31/3827301866.py:2: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=CFG_PATH):


In [21]:
# paths
cfg['data']["paths"]['train_csv'] = "/kaggle/input/birdclef2025-cv-split/cv_split.csv"
cfg['inference']['train_audio_path'] = "/kaggle/input/birdclef-2025/train_audio"
cfg['inference']['test_soundscape_path'] ="/kaggle/input/birdclef-2025/test_soundscapes"

cfg['inference']['batch_size'] = 2048

In [22]:
# checkpoint
artifact_base_path = "/kaggle/input/birdclef2025-models"
experiment = "6qiemv31_efficientnet_b0_softlabels_ce_softmax_augmentations"

model = 'model.ckpt'
train_cfg = 'config.yaml'

cfg['inference']['checkpoint_path'] = os.path.join(artifact_base_path, experiment, model)


In [23]:
# test on train soundscapes
# cfg['inference']['test_soundscape_path'] ="/kaggle/input/birdclef-2025/train_soundscapes"
# cfg['inference']['test_soundscape_path']

'/kaggle/input/birdclef-2025/train_soundscapes'

# Inference

In [25]:
from src.scripts.inference import main
main(cfg)